In [1]:
import pandas as pd
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [2]:
def db_new(patient_no , look_back):
    data = pd.read_csv("data/merged_patient" + str(patient_no-1) + ".csv")
    glucose = data["expected_glucose"]
    tt = []
    for i in range(look_back):
        tt.append([])
    
    rows = data.shape[0]
    for i in range(look_back , rows):
        for j in range(1,look_back+1):
            tt[j-1].append(glucose[i-j])

    to_drop = [i for i in range(look_back)]
    
    data = data.drop(to_drop);
    for i in range(1 , look_back+1):
        data[str("t-") + str(i)] = tt[i-1];

    return data
# def db_new(patient_no , look_back):
#     data = pd.read_csv("data/merged_patient" + str(patient_no-1) + ".csv")
# #     glucose = data["expected_glucose"]
# #     tt = []
# #     for i in range(look_back):
# #         tt.append([])
    
# #     rows = data.shape[0]
# #     for i in range(look_back , rows):
# #         for j in range(1,look_back+1):
# #             tt[j-1].append(glucose[i-j])

# #     to_drop = [i for i in range(look_back)]
    
# #     data = data.drop(to_drop);
# #     for i in range(1 , look_back+1):
# #         data[str("t-") + str(i)] = tt[i-1];

#     return data
        
    

In [3]:
start = 1;
########3
look_back = 8
########
df = db_new(start,look_back)
df["pat_no"] = [1] * df.shape[0]

endlimit = 40
#patient_data = pd.read_csv("test.csv") 
#no_rows = patient_data.shape[0]
for i in range(start+1,endlimit+1):
    temp_df = db_new(i,look_back)
    temp_df["pat_no"] = [i] * temp_df.shape[0] #augmenting the patient no. as seperate column
    df = pd.concat([df , temp_df ])
    


In [4]:
def onehot(data):
    values = array(data)

    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    return onehot_encoded
def augment_onehot(encoded , df , prefix):
    
    for i in range(encoded.shape[1]):
        df[prefix+str(i)] = encoded[:,i]
    return df

In [5]:
onehot_rtg = onehot(list(df["RTG"]))      
onehot_rf = onehot(list(df["RF"]))   
onehot_liver = onehot(list(df["liver"]))
onehot_peripheral = onehot(list(df["peripheral"]))
onehot_par1 = onehot(list(df["par1"]))
onehot_par2 = onehot(list(df["par2"]))
df = augment_onehot(onehot_rtg , df , "rtg")
df = augment_onehot(onehot_rf , df , "rf")
df = augment_onehot(onehot_liver , df , "liver")
df = augment_onehot(onehot_peripheral , df , "peripheral")
df = augment_onehot(onehot_par1 , df , "par1")
df = augment_onehot(onehot_par2 , df , "par2")
df = df.drop(["RTG","RF","liver","peripheral","par1","par2"] , axis=1)



C:\Users\ayush\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\ayush\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you us

In [6]:
df = df.drop(["Name"] , axis=1)



In [7]:
to_ignore = ["expected_glucose" ,"pat_no" ]
#for i in range(1,look_back+1):
    #to_ignore.append("t-" + str(i))

for var in df.keys():
    try:
        if(var not in to_ignore):
            if((df[var].max()-df[var].min())!=0):
                df[var] = ((df[var]-df[var].mean())/(df[var].std()))
    except:
        print(var , "created problem")

In [8]:
df.to_csv('feedable(new)(verion-2)(lb-' + str(look_back) + '.csv', index=False)


In [9]:
df["pat_no"]

8      1
9      1
10     1
11     1
12     1
      ..
92    40
93    40
94    40
95    40
96    40
Name: pat_no, Length: 3560, dtype: int64